<a href="https://colab.research.google.com/github/DylanCTY/TextAnalytics_LearningSpace/blob/main/IB9CW0_5504008_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Ollama

In [ ]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10941    0 10941    0     0  28607      0 --:--:-- --:--:-- --:--:-- 28566
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Get Ollama running in the background of the Colab

In [ ]:
# Set up Ollama as a global variable and run it in the background
OLLAMA_MODEL='phi:latest'
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # Verify the global variable is set

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # Print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 437
ollama version is 0.1.41


Query the model to generate text about football games in the UK from 2004 to 2005

In [ ]:
# Query the model via the command line
!ollama run $OLLAMA_MODEL "Briefly introduce the football games in the UK from 2004 to 2005."

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕

Start to build our RAG. Firstly, this involves installing a bunch of packages:

In [ ]:
# Install necessary packages
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
 

Read in the text and implement the chunking

In [ ]:
# Ensure the necessary packages are installed
!pip install llama-index llama-index-embeddings-huggingface

from zipfile import ZipFile
import os
from pathlib import Path
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from transformers import AutoModel, AutoTokenizer
from numpy.linalg import norm
import torch
from llama_index.core.data_structs.node import TextNode

In [ ]:
# Unzip the dataset
zip_file_path = '/content/bbcsport_football.zip'  # Path to the uploaded zip file
unzip_dir = '/content/bbcsport_football'
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# Combine the content of all text files into a single string, excluding hidden and system files
combined_text = ""
for root, dirs, files in os.walk(unzip_dir):
    for filename in sorted(files):
        if filename.endswith(".txt") and not filename.startswith('._') and not filename.startswith('__MACOSX'):
            with open(os.path.join(root, filename), 'r', encoding='utf-8', errors='ignore') as file:
                combined_text += file.read() + "\n"

# Replace "\n" (paragraph break) and "\t" (tab character)
combined_text = combined_text.replace("\n", "")
combined_text = combined_text.replace("\t", "")

# Write the combined text to a temporary file
temp_file_path = '/content/combined_bbcsport.txt'
with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
    temp_file.write(combined_text)

# Read the combined text to verify its content
with open(temp_file_path, 'r', encoding='utf-8') as temp_file:
    content = temp_file.read()
    print(f"Combined text length: {len(content)} characters")
    print(f"First 1000 characters of the combined text:\n{content[:1000]}")

Combined text length: 516804 characters
First 1000 characters of the combined text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete co

1-1. Semantic Chunking (model: BAAI/bge-small-en-v1.5)

In [ ]:
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=93, embed_model=embed_model
)

# Get nodes using the semantic chunking model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Check if bbcsport_nodes is defined and not empty
print(f"Total nodes created: {len(bbcsport_nodes)}")

# Print the first few nodes to verify their content
for i in range(min(5, len(bbcsport_nodes))):
    print(f"Node {i+1} text:\n{bbcsport_nodes[i].text}\n")

Combined text length: 516804 characters
First 1000 characters of the combined text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete co

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total nodes created: 402
Node 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Good

1-2. Semantic Chunking (model: jinaai/jina-embeddings-v2-small-en)

In [ ]:
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=93, embed_model=embed_model
)

# Get nodes using the semantic chunking model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Check if bbcsport_nodes is defined and not empty
print(f"Total nodes created: {len(bbcsport_nodes)}")

# Print the first few nodes to verify their content
for i in range(min(5, len(bbcsport_nodes))):
    print(f"Node {i+1} text:\n{bbcsport_nodes[i].text}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Total nodes created: 282
Node 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Good

In [ ]:
#Trial

# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=93, embed_model=embed_model
)

# Get nodes using the semantic chunking model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Convert nodes to the expected document type (TextNode)
docs = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes)]

# Check if docs is defined and not empty
print(f"Total docs created: {len(docs)}")

# Print the first few docs to verify their content
for i in range(min(5, len(docs))):
    print(f"Doc {i+1} text:\n{docs[i].text}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias

NameError: name 'TextNode' is not defined

2. Sentence Splitter

In [ ]:
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# We will limit to chunk size 100
parser = SentenceSplitter(chunk_size=200, chunk_overlap=50)

# Get nodes using the sentence splitting model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Check if bbcsport_nodes is defined and not empty
print(f"Total nodes created: {len(bbcsport_nodes)}")

# Print the first few nodes to verify their content
for i in range(min(5, len(bbcsport_nodes))):
    print(f"Node {i+1} text:\n{bbcsport_nodes[i].text}\n")

Check the output

In [ ]:
# Access the second node's text
print(bbcsport_nodes[1].text)

He's persistent and never gives in. "I don't know how many fouls he had He gets up and wants the ball again, he's truly a fabulous player." Everton: Martyn, Hibbert, Yobo, Stubbs, Naysmith, Osman, Carsley, Arteta, Kilbane, McFadden, Bent. Subs: Wright, Pistone, Weir, Plessis, Vaughan. 


Use Llama Index's SimpleDirectoryReader to load all the documents. After this we will specify an embedding model from HuggingFace and add all of this to Llama Index's settings

Model 1

In [ ]:
# Use the combined text file from the previous steps
combined_file_path = "/content/combined_bbcsport.txt"
reader = FlatReader()
docs = reader.load_data(Path(combined_file_path))
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 1 docs


Model 2

In [ ]:
# Trial
# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 282 docs


In [ ]:
# Use the combined text file from the previous steps
combined_file_path = "/content/combined_bbcsport.txt"
reader = FlatReader()
docs = reader.load_data(Path(combined_file_path))
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 1 docs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias

Setup a vector database (this time using Chroma DB) and index our documents (via embeddings). This will allow us to search for relevant documents to our query

In [ ]:
#Trial
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-bbcsport") in the db
chroma_collection = db.create_collection("demo-for-bbcsport")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs,  # the abstract documents (nodes from bbcsport_nodes)
    storage_context=storage_context,
    embed_model=embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

AttributeError: 'TextNode' object has no attribute 'get_doc_id'

In [ ]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-bbcsport") in the db
chroma_collection = db.create_collection("demo-for-bbcsport")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs, # the abstract documents (e.g. "Output0.txt")
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

name='demo-for-bbcsport' id=UUID('a5205320-5f32-47e0-9fd3-53cfb8254478') metadata=None tenant='default_tenant' database='default_database'
Collection name is: demo-for-bbcsport


Prompt Template

In [ ]:
# Set up a prompt template to ensure the LLM always answers
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
    ]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("Who is Thierry Henry?")
)

 Thierry Henry is a professional footballer who has played for many clubs including Arsenal in England and Paris Saint-Germain in France. He is known for his speed, agility, and scoring ability on the field. He has also represented the French national team at various levels.



Thierry Henry, a famous footballer, has played for several teams, including Arsenal (in England) and Paris Saint-Germain (in France). 

Assume that Thierry Henry can play in two positions: forward (F) or midfielder (M). He scored 4 goals in his most recent game. However, we don't know which position he played in the last game. The following clues are given:
1. If Thierry plays as a forward in the next match, then he will score at least one goal again. 
2. If Thierry is playing as a midfielder in the next game, then he won't score any goals.
3. Thierry cannot play both positions in the same game.

Question: In which position does Thierry Henry need to play in his next game to guarantee that he scores at least one

Start retrieval augmented generating

In [ ]:
# Assuming the necessary steps to create an index have been done
# Initialize query engine and run sample queries
query_engine = index.as_query_engine()  # use the vector db for queries

# Sample queries for testing
queries = [
    "Who won the crown of the Premier League in 2005?",
    "Describe the key players in Manchester United during the 2004-2005 season.",
    "Who is the leading goal scorers in the Premier League during the 2004-2005 season?",
    "How did Chelsea perform in the 2004-2005 football season?",
    "Provide an overview of the UEFA Champions League matches involving UK teams in 2004-2005.",
    "Detail the performance of Liverpool in the 2004-2005 season."
]

# Execute and print responses for each query
for query in queries:
    response = query_engine.query(query)  # query the model with context
    print(f"Query: {query}")
    print(f"Response: {response.response}\n")

Query: Who won the crown of the Premier League in 2005?
Response:  Based on the given context information, we can assume that Chelsea is leading Manchester United by six points with four games remaining in the season. However, it would be incorrect to state that they have a 100% chance of winning the title as other factors such as the performance of their opponents and their own team's performance are yet to play out.


Query: Describe the key players in Manchester United during the 2004-2005 season.
Response:  Given the context information and not prior knowledge, answer the query.


Query: Who is the leading goal scorers in the Premier League during the 2004-2005 season?
Response:  At this point of time, I do not have access to real-time data or statistics. However, based on my database, one can get information about the top goalscorer in the Premier League for every season from the year 1900s until now. The Premier League is a professional football league in England and it consists 

Inspect the output metadata

In [ ]:
response.metadata # print the full output from the LLM

{'bf0387c6-366e-4e1e-81d8-dd71692e0da9': {'filename': 'combined_bbcsport.txt',
  'extension': '.txt'},
 '081ea325-82a5-4844-a79d-b9cb4840461a': {'filename': 'combined_bbcsport.txt',
  'extension': '.txt'}}